<a href="https://colab.research.google.com/github/xhdixhfl/Goggles_project/blob/main/review_infomation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 드라이브 마운팅
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# library loading

In [ ]:
 #우분투 업데이트-> 크롬 드라이버 설치 snap
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.scgM7vKhGY/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.pQHSDVVlPV/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.UPX2WiytMH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [ ]:
# 필수 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time
import re # 정규표현식
import pandas as pd #csv저장
from datetime import datetime # 시간 체크
from pytz import timezone

# page로딩과 구조 로딩에 gab이 있어서 원하는 tag나올떄까지 기다리는거
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# driver setting

In [ ]:
# 화면 주소 정의
## 스킨케어 토너
aa = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'
## 스킨케어 에센스
ab = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010009&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'
## 스킨케어 미스트
ac = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010010&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'

## 립메이크업
ba = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020006&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'
## 베이스메이크업
bb='https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020001&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'
## 아이메이크업
bc='https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020007&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0'

In [ ]:
#  driver setting
URL = 'https://www.oliveyoung.co.kr/store/main/main.do?oy=0' # 첫화면
ops = webdriver.ChromeOptions() 
ops.add_experimental_option('excludeSwitches', ['enable-logging'])
ops.add_argument('--headless')        # Head-less 설정
ops.add_argument('--no-sandbox')
ops.add_argument('--disable-dev-shm-usage')
# ops.add_argument('__headlees') ## 확인해보자

# chromedriver path
driver = webdriver.Chrome('chromedriver', options=ops)

# defined function

In [ ]:
# review count
def rv_num2():
  global q # 총 리뷰수
  global pn # 리뷰 페이지수
  global pc # 마지막 페이지의 리뷰수
  q = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text[1:-1]
  q = int(re.sub(r"[^0-9]", "", q))
  if q == 0:  # 리뷰수가 없는 경우
      pn = 'zero'
      pc = 'zero'
  elif 0 < q <= 10:  # 리뷰수가 10이하인 경우
      pn = 'one'
      pc = q
  else: # 리뷰수가 11이상
    # page_num(리뷰 페이지 수), product_count(마지막페이지의 리뷰수)
      pn = (q // 10) + 1 
      pc = q % 10

In [ ]:
## 리뷰의 키워드를 반환해주는 함수

def find_kw(i):
    try:  # css가 특징이 정확함
        x = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample ').text
    except:
        x = 'no keyword' # 키워드 선택을 안하는 경우
    finally:
        kw_l.append(x)

In [ ]:
## review text
def find_text(i):   # //*[@id="gdasList"]/li[1]/div[2]/div[2]/p   헤드 없고 키워드 없고 내용만  
        # 키워드 없는 헤드 //*[@id="gdasList"]/li[6]/div[2]/p    ## //*[@id="gdasList"]/li[6]/div[2]/div[2]   # 내용인데 헤드가 있으면
    try:
        text = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
    except:
        try:  # 리뷰 아우터만 존재하는 경우
            text = driver.find_element(By.CSS_SELLECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > p').text
        except:
            text = 'no text'  # 해당 리뷰 구조가 없으면 리뷰가 없는걸로
    finally:
        rv_l.append(text)

In [ ]:
## 한페이지 안에 가져올 리뷰정보를 반환하는 함수
def list_up(n=10):  # 파라미터는 한 페이지 안에 있는 리뷰수
    for i in range(1, n+1):
        time.sleep(0.1)
        x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]/div[1]/span[1]/span').text
        x = list(x)
        star = int(x[-2])
        star_l.append(star)
        find_kw(i)   # keywords function
        find_text(i) # 리뷰 함수
        # print('ok')

In [ ]:
## 한 상품에서 가져올 정보를 반환하는 함수(최대한 많이)
def one_product2(pn, pc):
     # 가장 첫페이지 리뷰가져오기
    list_up()
    print('1_page')
    for l in range(2, pn+1): # 리뷰 페이지 수 만큼 
        if str(l)[-1] == '0':
            try:  # 10, 20 등 뒤에 다음 버튼이 오는 경우
                driver.find_element(By.LINK_TEXT, f'{l}').click() # 페이지 넘검
                moment
                list_up() # 리뷰내용 가져오는 함수(default = 10)
                print(l,'page')
                driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click() # 다음 페이지로
                moment
                a = driver.find_elements(By.CLASS_NAME, 'review_cont') # 해당 페이지의 리뷰수 카운트
                if len(a) == 10: # 리뷰수 10개
                    list_up()
                else: 
                    list_up(pc) # pc랑 a가 같은 수임
                print(l,'page')
            except: # 클릭후 다음 버튼이 없는 경우
                driver.find_element(By.LINK_TEXT, f'{l}').click() # 마지막 페이지
                moment
                a = driver.find_elements(By.CLASS_NAME, 'review_cont') # 해당 페이지의 리뷰수 카운트
                list_up(len(a))  # pc가 0일 수도 있어서
                print(l, 'page')
                
        elif str(l)[-1] == '0' and l == pn: # 다음페이지가 마지막인 경우
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            list_up()
            print(l,'page')
            driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
            moment
            if pc != 0: # pc가 0이면 for문 안돌아가서 
                list_up(pc)
                print(l,'page^')
            else:
                list_up()
                print(l,'page')
        elif str(l)[-1] == '1': 
            print(l) # 11,21 등의 페이지는 버튼클릭이 다음 버튼이라 넣을 내용이 없음.
        elif l == (pn): # 현재가 마지막 페이지
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            if pc != 0:
                list_up(pc)
                print(l,'page')
            else:
                list_up() # 0으로 끝나면 10이니
                print(l,'page')
            # 끝자리수만큼 포문 pc list_up(pc)
            # print(l)
        elif '2' <= str(l)[-1] <= '9' and l != pn: # 0,1 마지막 페이지가 아닌경우 다음버튼 누른후 10개씩 가져옴
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            moment
            list_up()
            print(l,'page')

In [ ]:
## 상품의 리뷰를 가져오는 함수
### 위 함수들의 진행하는데 있어 생각하지 못한 오류 들이 발생할 수 있어서 예외처리하고자 함.
def try_pro():
  try:
      rv_num2()
      # rv_brand()  # 새로 추가됨(브랜드, 가격)
      if pn == 'zero' and pc == 'zero':
            # 상품리뷰 없음. 
          print('no')
          # non_l.append(pname) # 상품명 리스트에는 안올라감
      elif pn == 'one':
          # 1~10사이
        list_up(pc) # 10개라도 여기선 pc가 q임 ㅎ
        # print(pname, '_수집')
      else:
        one_product2(pn, pc)     # one_product2(pn, pc)
        # print(pname, '_수집')
  except:
        # pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
      # fail_l.append(pname)
      print("??? 문제 발생 ???")

In [ ]:
def one_cate():
     # 카테고리별 48개의 상품 클릭하기
      try:
          for i in range(1,13):
              for j in range(1,5):
                    # 리뷰 48개로 선택
                driver.find_element(By.XPATH, '//*[@id="Contents"]/div[5]/div[2]/ul/li[3]/a').click()
                driver.implicitly_wait(5)
                    # 상품 선택
                driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
                driver.implicitly_wait(5)
                # 리뷰칸 선택
                driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
                moment
                try_pro()
                driver.back()
                driver.implicitly_wait(5)
      except:
            print('___next___')


# action

In [ ]:
# page moving
driver.get(url=aa)
driver.implicitly_wait(5)

# 리뷰 컨테이너 요소를 만나기 위한 작업(explicitly wait지정)
driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{2}]/li[{1}]/div/a/img').click()
driver.implicitly_wait(5)
driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
## 리뷰컨테이너를 만날때까지 최대 6초 기다림
moment = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, 'review_cont'))) # 해당요소를 만나야만 가능

# 첫페이지 화면으로
driver.back()

In [ ]:
# 리스트 정의
# stored information
star_l = []
kw_l = []
rv_l = []

non_l = []
fail_l = []

# 링크 저장
link_l = [aa, ab, ac, ba, bb, bc]


In [ ]:
print('시작 시간: ',datetime.now(timezone('Asia/Seoul')))

시작 시간:  2023-01-31 09:08:52.458158+09:00


In [ ]:
# # 48개의 상품들의 정보를 10페이지 수
# for x in range(2,11):
#     try:
#         for i in range(1, 13):
#             for j in range(1,5):
#                 # 리뷰 48개로 선택
#                 driver.find_element(By.XPATH, '//*[@id="Contents"]/div[5]/div[2]/ul/li[3]/a').click()
#                 driver.implicitly_wait(5)
#                 # 상품 선택
#                 driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
#                 driver.implicitly_wait(5)
#                 # 리뷰칸 선택
#                 driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
#                 moment
#                 try_pro()
#                 driver.back()
#                 driver.implicitly_wait(5)
#     except:
#         print('어디서 오류가 났으려나? 일단 pass^^')
#     try:
#         driver.find_element(By.LINK_TEXT, f'{10+x}').click()
#         moment
#     except:
#         continue

In [ ]:
# 카테고리별로 48개의 상품 리뷰들을 최대한 많이 크롤링
for link in link_l:
    driver.get(url=link)
    driver.implicitly_wait(5)
    one_cate()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
31
32 page
33 page
34 page
35 page
36 page
37 page
38 page
39 page
40 page
40 page
41
42 page
43 page
44 page
45 page
46 page
47 page
48 page
49 page
50 page
50 page
51
52 page
53 page
54 page
55 page
56 page
57 page
58 page
59 page
60 page
??? 문제 발생 ???
1_page
2 page
3 page
4 page
5 page
??? 문제 발생 ???
1_page
2 page
3 page
4 page
??? 문제 발생 ???
1_page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
9 page
10 page
10 page
11
12 page
13 page
14 page
15 page
16 page
17 page
18 page
19 page
20 page
20 page
21
22 page
23 page
24 page
25 page
26 page
27 page
28 page
29 page
30 page
30 page
31
32 page
33 page
34 page
35 page
36 page
37 page
38 page
39 page
40 page
40 page
41
42 page
43 page
44 page
45 page
46 page
47 page
48 page
49 page
50 page
50 page
51
52 page
53 page
54 page
55 page
56 page
57 page
58 page
59 page
60 page
60 page
61
62 page
63 page
64 page
65 page
66 page
67 page
68 page
69 page
70 page
70 page
71
72 page
73 page
74 page
75 page
76 pag

In [ ]:
# driver.close()

df = pd.DataFrame({'star':star_l,'keyword':kw_l, 'rv':rv_l})
df.to_csv('/content/drive/MyDrive/data_RV2.csv', sep = ',', encoding = 'utf-8-sig', index=False)  # 카테고리 포함 버전

In [ ]:
print('끝난 시간: ',datetime.now(timezone('Asia/Seoul')))

끝난 시간:  2023-01-31 15:54:08.565431+09:00


In [ ]:
driver.close()

In [ ]:
import pandas as pd